<div class="alert alert-info" role="alert">
    <H1> Introduction </H1>
<p>
    This notebook contains the steps I took to analyze and label the WiDS 2018 Datathon data. The data contained demographic and behavioral information from a representative sample of survey respondents from India and their usage of traditional financial and mobile financial services. The dataset is a product of InterMedia’s research to help the world’s poorest people take advantage of widely available mobile phones and other digital technology to access financial tools and participate more fully in their local economies. 
    To obtain the data contact Intermedia directly at http://finclusion@intermedia.org and fill out a data request form [here](http://finclusion.org/data_fiinder/)
</p>

<p>

The goal of this datathon was to determine if a survey respondent was male or female (0 or 1), based on how they answered questions.

I performed the following steps to produce a model with a resulting accuracy of 0.96921 when applied to the test data, placing 73/231.

    <li>Wrangling the data</li>
    <li>Feature selection</li>
    <li>Optimization of an XGBoost model for predicting labels on a test dataset</li>
    
</p>
</div>

In [1]:
# general imports
import pickle
import pandas as pd
import numpy as np
import matplotlib.style
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.style.use('seaborn-deep')# load data into pandas dataframe

#imports for chi-squared
from scipy.stats import chi2_contingency
from collections import defaultdict

# imports for xgboost
import xgboost as xgb
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import DMatrix
from xgboost import cv
from sklearn.model_selection import GridSearchCV


C:\Users\aregel\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


<div class="alert alert-info" role="alert">
    <H1> Data Wrangle </H1>
<p>
    In order to build an effective model the data needs to be cleaned and orgainized. 
    
</p>

<p>
    <li>Read data into memory as a pandas dataframe</li>
    <li>Remove empty columns</li>
    <li>Insure feature agreement between test and training data</li>
    <li>Separate the different data types and cast the categorical data as object type</li>
    
</p>
</div>

In [2]:
# import training data
df_train = pd.read_csv(r'train.csv', low_memory=False)
df_test = pd.read_csv(r'test.csv', low_memory=False)

In [3]:
# remove all columns with no data (there are no rows with all NaNs)
df_nona_tr = df_train.dropna(axis=1, how='all')
df_nona_ts = df_test.dropna(axis=1, how='all')
# create list of columns both datasets have
comb_cols = list(set(df_nona_tr) & set(df_nona_ts))
# define cleaned datasets
clean_train = df_train[comb_cols]
clean_test = df_test[comb_cols]

##### Separate different data types

In [4]:
# Isolate text data
text_train = clean_train.drop(columns=['LN2_RIndLngBEOth','LN2_WIndLngBEOth']).select_dtypes(exclude=['float64','int64'])
text_test = clean_test.drop(columns=['LN2_RIndLngBEOth','LN2_WIndLngBEOth']).select_dtypes(exclude=['float64', 'int64'])


The data dictionary provided a description of all of the catagorical data, so I will use those column names to separate the catagorical data from the numerical

In [5]:
# Create list of catagorical feature names
data_dictionary = pd.read_excel('WiDS data dictionary v2.xlsx')
col_list = list(data_dictionary['Column Name'][1:].apply(lambda x: str(x)))
col_list.append(['LN2_RIndLngBEOth','LN2_WIndLngBEOth'])
categorical_column_names = [name for name in comb_cols if name in col_list]
# Cast catagorical data as object datatype
categorical_train = clean_train[categorical_column_names].drop(columns='DG1').astype('object')
categorical_test = clean_test[categorical_column_names].drop(columns='DG1').astype('object')

In [6]:
# Dataframe of numerical data
drop_columns = categorical_column_names + list(text_train)
numerical_train = clean_train.drop(columns=drop_columns)
numerical_test = clean_test.drop(columns=drop_columns)

#### Verify data type separation

In [7]:
print(clean_train.info())
print(len(list(numerical_train)) + len(list(categorical_train)) + len(list(text_train)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18255 entries, 0 to 18254
Columns: 1168 entries, IFI16_5 to MM42_3
dtypes: float64(844), int64(237), object(87)
memory usage: 162.7+ MB
None
1167


<div class="alert alert-success">
<H3> Conclusions for Data Wrangle </H3>

<p>
The separate columns sum to the total number of columns -1, which makes sense as the labeled column is not included in any of the catagories
</p>
</div>

<div class="alert alert-info" role="alert">
    <H1> Feature Selection </H1>

<p>
    <li>Remove any column that is more than 60% NaN</li>
    <li>Use the Chi Squared metric to determine if the categorical data is dependent on gender</li>
    <li>One-hot encode the categorical data</li>
 
</p>
</div>

In [8]:
# Transform dataframes where NaN = 0 and value=1, and sum them
text_count = text_train.notna().astype(int).sum()
categorical_count = categorical_train.notna().astype(int).sum()
numerical_count = numerical_train.notna().astype(int).sum()
# Define threshold for 40% filled in
threshold = 18255*0.4

# Create list of columns that exceed the threshold
valid_text_columns = []
valid_categorical_columns = []
valid_numerical_columns = []
for text_name, text_num, categorical_name, categorical_num,numerical_name, numerical_num in zip(text_count.index, text_count,categorical_count.index,categorical_count,numerical_count.index, numerical_count):
    if text_num > threshold:
        valid_text_columns.append(text_name)
    if categorical_num > threshold:
        valid_categorical_columns.append(text_name)
    if numerical_num > threshold:
        valid_numerical_columns.append(text_name)

    

In [9]:
# print the number of valid columns for each datatype
print(len(valid_text_columns))
print(len(valid_categorical_columns))
print(len(valid_numerical_columns))

0
14
13


In [10]:
# see how many valid columns there are with a threshold of only 10% of rows filled

# Transform dataframes where NaN = 0 and value=1, and sum them
text_count = text_train.notna().astype(int).sum()
categorical_count = categorical_train.notna().astype(int).sum()
numerical_count = numerical_train.notna().astype(int).sum()
# Define threshold for 10% filled in
threshold = 18255*0.1

# Create list of columns that exceed the threshold
valid_text_columns = []
valid_categorical_columns = []
valid_numerical_columns = []
for text_name, text_num, categorical_name, categorical_num,numerical_name, numerical_num in zip(text_count.index, text_count,categorical_count.index,categorical_count,numerical_count.index, numerical_count):
    if text_num > threshold:
        valid_text_columns.append(text_name)
    if categorical_num > threshold:
        valid_categorical_columns.append(text_name)
    if numerical_num > threshold:
        valid_numerical_columns.append(text_name)

In [11]:
print(len(valid_text_columns))
print(len(valid_categorical_columns))
print(len(valid_numerical_columns))

0
21
31


<div class="alert alert-warning">
<p>
<H5> Eliminating almost 98% of all the features will probably lead to underfitting when predicting on new data.  However, the text data is especially incomplete, as none of the columns are even 10% complete. Thus, I will not use any of the text data for training the XGBoost model and will use the chi-squared metric to evaluate the categorical data. </H5>
</p>
</div>

#### Filter out the categorical data not dependent on gender, using the chi-squared test statistic with contingency tables

In [12]:
# Helper functions

# function to count values for each possible category
def cat_count(pd_series):
    categories = list(set(pd_series))
    cat_count = dict.fromkeys(categories, 0)
    for cat in pd_series:
        cat_count[cat] += 1
    return cat_count

# function to create joint dist table
def joint_dist_table(cat_series):
    # split male and female counts, and drops and Nans
    F_series = cat_series[categorical_train.is_female == 1].dropna()
    M_series = cat_series[categorical_train.is_female == 0].dropna()
    # find possible categories for each column and gender
    F = cat_count(F_series)
    M = cat_count(M_series) 
    # only use the categories that exist for both genders
    keep = set(F) & set(M)
    F_new = {k: F[k] for k in keep}
    M_new = {k: M[k] for k in keep}
    # combine counts in dataframe and make distribution table
    dist_table = pd.DataFrame.from_dict(F_new, orient='index')
    dist_table[1] = M_new.values()
    # format the distribution table
    final_dist_table = dist_table.rename(columns={0:'Male',1:'Female'}).transpose()
    return final_dist_table 

In [13]:
# add the label to the categorical dataframe
categorical_train['is_female'] = df_train.is_female

In [14]:
# determine p-value of the chi-squared metric for each categorical column
chi_dict = defaultdict(list)
for cat_cols in list(categorical_train):
    try:
        # create joint distribution table
        jd_table = joint_dist_table(categorical_train[cat_cols])
        chi_test_value, chi_p, degfree, exp_val = chi2_contingency(jd_table)
        chi_dict[cat_cols] = [chi_test_value, chi_p, degfree, exp_val]
    except ValueError:
        chi_dict[cat_cols] = [0,0,0,0]

In [15]:
# filter columns according to significance level
sig_level = 0.01
sig_cols = []
for k,v in chi_dict.items():
    if v[1] < sig_level:
        sig_cols.append(k)

In [16]:
# remove the label column from the significant features list
sig_cols.remove('is_female')

In [17]:
# create a dataframe for only the categorical data dependent on gender
significant_categorical_data = df_nona_tr[sig_cols].astype('object') # cast to object type
encoded_categorical_data = pd.get_dummies(significant_categorical_data, dummy_na=True) # one-hot encode


In [18]:
# count how many significant columns are left
print(len(encoded_categorical_data.columns))

2135


<div class="alert alert-info" role="alert">
    <H1> Optimize XGBoost model </H1>
        <p>
            <li>Determine if using the categorical, numerical, or a combination of the data improves the model</li>
            <li>Use cross-fold validation and gridsearch to optimize hyper-parameters</li>
            <li>Validate optimized parameters</li>
            <li>Predict label for test data</li>
        </p>
    

<div class="alert alert-warning">
<H7> I'm using the binary logistic loss function because it returns the probability of the label being 1, which is a requirement of the datathon. I will be optimizing using accuracy as that is the metric used for the datathon.</H7>
</div>

In [19]:
# Use unoptomized xgboost model with numerical, categorical, and a combination of the two 

In [20]:
# Test, train, split
# GridSearch


In [21]:
# Train model with all of the data

In [22]:
# Process test data
    # filter significant columns
    # hot-encode

In [ ]:
# Predict label of test data with optimized model

In [ ]:
# create features(X), and target(y)
X = encoded_categorical_data
y = df_train['is_female']

In [ ]:
# split data for testing and trainning
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=123)

In [ ]:
fixed_parameters = {
               'max_depth':4,
               'learning_rate':0.3,
               'min_child_weight':3,
               'colsample_bytree':0.85,
               'subsample':0.85,
               'gamma':0,
               'max_delta_step':0,
               'colsample_bylevel':1,
               'scale_pos_weight':1,
               'base_score':0.5,
               'seed':5,
               'objective':'binary:logistic',
               'silent': 1}

In [ ]:
xg_reg = xgb.XGBRegressor(**fixed_parameters)

In [ ]:
xg_reg.fit(X, y)

In [ ]:
y_pred = xg_reg.predict(X)

In [ ]:
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy_score(y, predictions)

In [ ]:
pickle.dump(xg_reg, open('xgb_final_model.dat', 'wb'))

In [ ]:
len(test_dummy)

In [ ]:
X_test = test_dummy[model_cols]

In [ ]:
len(X_test)

In [ ]:
y_test = xg_reg.predict(X_test)

In [ ]:
len(y_test)

In [ ]:
df_nona_ts.test_id.values

In [ ]:
df_dict = {'test_id': df_nona_ts.test_id.values, 'is_female':y_test}

In [ ]:
g = pd.DataFrame(df_dict)

In [ ]:
g.to_csv('sub4.csv')